# 천안정수장 원수 및 침전탁도

필요한 라이브러리 import

In [1]:
import streamlit as st
import pandas as pd
import seaborn as sns
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import openpyxl

plt.rcParams['font.family'] = 'Malgun Gothic'  # 또는 출력된 다른 이름
plt.rcParams['axes.unicode_minus'] = False  # 한글 폰트 사용 시, 마이너스 기호 깨짐 방지
%matplotlib inline


### 로그 변환할지 확인하기 위한 히스토그램 출력

In [ ]:
def check_to_log(df):
    # 로그 변환 대상 컬럼 리스트 (양수여야 함)
    columns_to_check = ['탁도', '약품주입율 계산']  # 필요시 다른 컬럼 추가

    # 히스토그램 출력
    for col in columns_to_check:
        fig, axs = plt.subplots(1, 2, figsize=(12, 4))
        
        # 원래 분포
        axs[0].hist(df[col].dropna(), bins=50, color='skyblue')
        axs[0].set_title(f'{col} - 원래 분포')
        
        # 로그 변환 분포
        log_data = df[col][df[col] > 0]  # 0 이하 제외
        log_data = np.log10(log_data)
        axs[1].hist(log_data.dropna(), bins=50, color='lightgreen')
        axs[1].set_title(f'{col} - 로그 변환 분포')

        plt.tight_layout()
        plt.show()

### 데이터 불러오기
### 및 데이터 전처리

In [ ]:

# 데이터 불러오기
df = pd.read_csv('./data/CA_Water_Quality.csv', encoding="CP949", low_memory=False)

# df["로그 탁도"] = np.log10(df["탁도"])
# df["로그 응집제 주입률"] = np.log10(int(df["약품주입율 계산"]))
# np.log10은 데이터 전처리 후 사용


df.rename(columns={'약품주입율 계산': '약품주입율'}, inplace=True)


df = df[
    [
        "탁도",
        "pH",
        "알칼리도",
        "전기전도도",
        "수온",
        "유입유량",
        "침전탁도",
        "약품주입율"
    ]
]




C:\Users\HB\AppData\Local\Temp\ipykernel_13044\3613894599.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/CA_Water_Quality.csv', encoding="CP949")


In [14]:

# 두 컬럼만 숫자로 변환 (숫자가 아닌 값은 NaN으로 처리)
df['탁도'] = pd.to_numeric(df['탁도'], errors='coerce')
df['약품주입율'] = pd.to_numeric(df['약품주입율'], errors='coerce')

# 변환 후 NaN 확인 (비정상값이 있었는지 확인 가능)
print(df[['탁도', '약품주입율']].isna().sum())

# 탁도 0개, 약품주입율 계산 11개 데이터가 na로, 행 삭제

df = df.dropna(subset=['탁도', '약품주입율'])

# 탁도 분포에 따라 로그 변환 실행
df['로그 탁도'] = np.log10(df["탁도"])


탁도        0
약품주입율    11
dtype: int64


c:\Users\HB\miniconda3\envs\KPro_HB\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [12]:
df_raw = df.copy()
df_raw.describe()

c:\Users\HB\miniconda3\envs\KPro_HB\lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,탁도,pH,알칼리도,전기전도도,수온,유입유량,침전탁도,약품주입율,로그 탁도
count,526970.000000,526970.000000,526970.000000,526970.00000,526970.000000,526970.000000,526970.000000,526970.000000,5.269700e+05
mean,31.184532,7.216701,30.193983,137.06379,14.891338,12005.192493,32.747157,22.972786,-inf
std,231.648163,0.828031,5.294893,22.13868,19.550480,1663.340725,341.258221,13.894182,NaN
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,-inf
25%,1.880000,7.090000,26.660000,119.28000,9.170000,11095.000000,0.490000,13.000000,2.741578e-01
50%,2.600000,7.360000,32.210000,146.41000,13.550000,11996.000000,0.690000,16.000000,4.149733e-01
75%,12.870000,7.460000,33.880000,153.44000,21.860000,13047.000000,1.000000,35.000000,1.109579e+00
max,2755.000000,7.710000,41.490000,172.09000,7814.380000,142090.000000,4954.510000,92.000000,3.440122e+00


## 예측 함수

In [56]:
def run(target, input, max_depth, n_estimators, learning_rate, subsample):

    Xt, Xts, yt, yts = train_test_split(input, target, test_size=0.2, shuffle=False)

    xgb = XGBRegressor(
        random_state=2, 
        n_jobs=-1,
        max_depth=max_depth,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        subsample=subsample
    )

    xgb.fit(Xt, yt)
    
    yt_pred = xgb.predict(Xt)
    yts_pred = xgb.predict(Xts)

    mse_train = mean_squared_error(10**yt, 10**yt_pred)
    mse_test = mean_squared_error(10**yts, 10**yts_pred)
    st.write(f"학습 데이터 MSE: {mse_train}")
    st.write(f"테스트 데이터 MSE: {mse_test}")

    r2_train = r2_score(10**yt, 10**yt_pred)
    r2_test = r2_score(10**yts, 10**yts_pred)
    st.write(f"학습 데이터 R2: {r2_train}")
    st.write(f"테스트 데이터 R2: {r2_test}")

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    ax = axes[0]
    print(Xt.columns)
    ax.scatter(Xt.iloc[:, 0], yt, s=3, label="학습 데이터 (실제)")
    ax.scatter(Xt.iloc[:, 0], yt_pred, s=3, label="학습 데이터 (예측)", c="r")
    ax.grid()
    ax.legend(fontsize=13)
    # ax.set_xlabel("로그 원수 탁도")
    # ax.set_ylabel("로그 응집제 주입률")
    
    ax.set_xlabel(input.columns[0])
    ax.set_ylabel(target.columns[0])

    ax.set_title(
        rf"학습 데이터  MSE: {round(mse_train, 4)}, $R^2$: {round(r2_train, 2)}",
        fontsize=18,
    )

    ax = axes[1]
    # ax.scatter(Xt["로그 원수 탁도"], yt, s=3, label="학습 데이터 (실제)")
    # ax.scatter(Xt["로그 원수 탁도"], yt_pred, s=3, label="학습 데이터 (예측)", c="r")
    
    ax.scatter(Xt.iloc[:, 0], yt, s=3, label="학습 데이터 (실제)")
    ax.scatter(Xt.iloc[:, 0], yt_pred, s=3, label="학습 데이터 (예측)", c="r")

    ax.grid()
    ax.legend(fontsize=13)

    ax.set_xlabel(input.columns[0])
    ax.set_ylabel(target.columns[0])

    ax.set_title(
        rf"테스트 데이터  MSE: {round(mse_test, 4)}, $R^2$: {round(r2_test, 2)}",
        fontsize=18,
    )

    st.pyplot(fig)

## 메인함수

In [ ]:
def main():

    # load data
    dff = load_data()

    # Select Variable
    st.markdown("## Select Target Variable")
    column = st.selectbox("Target 변수를 선택하세요.", dff.columns.tolist())
    col = dff[[column]]
    st.dataframe(dff[[column]].head())

    st.markdown("## Select Input Variables")
    input_columns = st.multiselect("복수의 컬럼을 선택하세요.", dff.columns.tolist())
    filtered_col = dff[input_columns]

    st.dataframe(dff[input_columns].head())

    # Hyperparameters
    max_depth = st.slider("Select max depth", min_value=0, max_value=20, value=3)
    n_estimators = st.slider("n_estimators", min_value=20, max_value=500, value=50)
    learning_rate = st.slider("learning_rate", min_value=0.00, max_value=1.00, step=0.01, value=0.1)
    subsample = st.slider("subsample", min_value=0.00, max_value=1.00, step=0.01, value=0.8)

    if st.button("차트 만들기"):
        run(col, filtered_col, max_depth, n_estimators, learning_rate, subsample)